In [6]:
import pandas as pd
import numpy as np
import os
import json
os.getcwd()

'/Users/nicholashamlin/Documents/Grad School/209 - Data Visualization/W209-3/Nick'

In [7]:
#Load raw data as pandas dataframe for easy munging
#f=open('Micro_Citizens_Round1_FINAL2-1 (1).csv','rb')
f=open('cleaned_survey_data.csv','rb')
survey_data=pd.read_csv(f)
f.close()
#load geojson as regular json (not pandas)
j=open('district_data.geojson','rb')
geo_data=json.load(j)
j.close()

In [8]:
for i in geo_data['features']:
    i['properties']['District']=i['properties'].pop('DISTRICT').title()

In [9]:
survey_data['District']=survey_data['District'].str.title()

In [10]:
def update_json(name_to_insert,grouped_data,data_to_update):
    """given a new column name, a grouped series, and json object, write new properties"""
    for i in data_to_update['features']:
        try:
            i['properties'][name_to_insert]=grouped_data[i['properties']['District']]
        except KeyError:
            #print i['properties']
            #print grouped_data[i['properties']['DISTRICT']]
            i['properties'][name_to_insert]=None

In [11]:
#Data Cleansing
#data['District']=data['District'].str.upper()
#data['Answer']=data['Are_your_main_problems_being_a'].str[0]

responses= {"1___not_at_all":1 ,"2___very_little" :2,"2___not_very_m" :2,"3___neutral":3,"don_t_know":None,"4___mostly_yes":4,"5___completely_yes":5,"refused ":None}

questions=['Are_your_main_problems_being_a','Do_you_feel_you_have_been_hear']

for q in questions:
    survey_data[q+'_processed'] = survey_data[q].replace(responses)
    # now convert the types
    survey_data[q+'_processed'] = survey_data[q+'_processed'].convert_objects(convert_numeric=True)
    
    #Aggregate results and save to geojson
    grouped=survey_data.groupby(['District'])
    grouped_answer=grouped[q+'_processed']

    output=grouped_answer.aggregate(np.mean)
    update_json(q,output,geo_data)

In [12]:
#Check one example
geo_data['features'][36]['properties']

{'Are_your_main_problems_being_a': 1.5252525252525253,
 'District': u'Sindhupalchok',
 'Do_you_feel_you_have_been_hear': 1.4204545454545454,
 u'needs': 1.525}

In [13]:
#Use this to lookup an index for a particular district
for k,i in enumerate(geo_data['features']):
    print str(k)+" "+i['properties']['District']

0 Humla
1 Darchula
2 Bajhang
3 Mugu
4 Bajura
5 Baitadi
6 Dolpa
7 Jumla
8 Kalikot
9 Doti
10 Dadeldhura
11 Achham
12 Mustang
13 Dailekh
14 Jajarkot
15 Kanchanpur
16 Kailali
17 Rukum
18 Surkhet
19 Manang
20 Myagdi
21 Gorkha
22 Bardiya
23 Salyan
24 Baglung
25 Kaski
26 Rolpa
27 Lamjung
28 Parbat
29 Rasuwa
30 Pyuthan
31 Dhading
32 Banke
33 Gulmi
34 Dang
35 Syangja
36 Sindhupalchok
37 Dolakha
38 Tanahu
39 Arghakhanchi
40 Solukhumbu
41 Nuwakot
42 Sankhuwasabha
43 Palpa
44 Taplejung
45 Chitwan
46 Nawalparasi
47 Kapilbastu
48 Ramechhap
49 Kathmandu
50 Rupandehi
51 Kavre
52 Bhaktapur
53 Makwanpur
54 Lalitpur
55 Okhaldhunga
56 Bhojpur
57 Parsa
58 Sindhuli
59 Khotang
60 Panchthar
61 Bara
62 Tehrathum
63 Rautahat
64 Dhankuta
65 Sarlahi
66 Udayapur
67 Mahottari
68 Ilam
69 Dhanusa
70 Siraha
71 Saptari
72 Morang
73 Sunsari
74 Jhapa


In [15]:
#save results
t=open('feedback_map.geojson','wb')
json.dump(geo_data,t)
t.close()